In [2]:
import pandas as pd
df = pd.read_csv(
'/Users/mavinjames/Projects/Basketball_Modeling/NCAAB_Sports_Reference_Scraper/data/merged_dataset.csv')

In [3]:
cols = [
    "ft_pct","opp_ft_pct","fg_pct","fg3a","fg3_pct","fg","fga","fg2a","fg3","ft","fta",
    "efg_pct","fg2_pct","orb","drb","opp_fg3","opp_fg_pct","blk","opp_fg","pf","tov",
    "ast","stl","trb","team_game_result","opp_efg_pct","opp_ft","opp_fta","opp_orb",
    "opp_drb","opp_trb","opp_ast","opp_stl","opp_blk","opp_tov","opp_fg3a","opp_fg3_pct",
    "opp_fg2","opp_fg2a","opp_fg2_pct","opp_fga","team_possessions","opp_possessions",
    "opp_pf","possessions","off_rtg","def_rtg","win","fg2","net_rtg",
    "team_game_num_season","opp_opp_team_game_score","team_game_score","score_diff",
    "opp_team_game_score"
]

df = df.dropna(subset=cols)

In [4]:
features = ['is_Home','cum_off_rtg',
 'cum_def_rtg',
 'cum_net_rtg',
 'win_pct_last_10',
 'efg_pct_last_5',
 'efg_pct_last_10',
 'avg_fta_last_5',
 'avg_fta_last_10',
 'avg_ast_last_5',
 'avg_ast_last_10',
 'avg_trb_last_5',
 'avg_trb_last_10',
 'avg_orb_last_5',
 'avg_orb_last_10',
 'avg_tov_last_5',
 'avg_tov_last_10',
 'avg_team_game_score_last_5',
 'avg_team_game_score_last_10',
 'avg_opp_team_game_score_last_5',
 'avg_opp_team_game_score_last_10',
 'avg_score_diff_last_5',
 'avg_score_diff_last_10',
 'avg_possessions_last_5',
 'avg_possessions_last_10',
 'rest_days',
 'opp_rest_days',
 'opp_win_pct_last_10',
 'opp_efg_pct_last_5',
 'opp_efg_pct_last_10',
 'opp_avg_fta_last_5',
 'opp_avg_fta_last_10',
 'opp_avg_ast_last_5',
 'opp_avg_ast_last_10',
 'opp_avg_trb_last_5',
 'opp_avg_trb_last_10',
 'opp_avg_orb_last_5',
 'opp_avg_orb_last_10',
 'opp_avg_tov_last_5',
 'opp_avg_tov_last_10',
 'opp_avg_team_game_score_last_5',
 'opp_avg_team_game_score_last_10',
 'opp_avg_opp_team_game_score_last_5',
 'opp_avg_opp_team_game_score_last_10',
 'opp_avg_score_diff_last_5',
 'opp_avg_score_diff_last_10',
 'opp_avg_possessions_last_5',
 'opp_avg_possessions_last_10',
 'opp_cum_off_rtg',
 'opp_cum_def_rtg',
 'opp_cum_net_rtg',
 'avg_score_comp_last_10',
 'efg_comp_last_10',
 'avg_tov_comp_last_10',
 'avg_orb_comp_last_10',
 'avg_fta_comp_last_10',
 'rest_days_comp', 'net_rtg_comp',
 'home_road_split_comp', "pace_mismatch_signed"]  # your cols

target = "score_diff"

corrs = df[features + [target]].corr()[target].sort_values(ascending=False)
corrs

score_diff                    1.000000
net_rtg_comp                  0.526028
is_Home                       0.349765
avg_score_diff_last_10        0.336407
cum_net_rtg                   0.330177
                                ...   
opp_avg_ast_last_10          -0.170005
opp_win_pct_last_10          -0.283551
opp_avg_score_diff_last_5    -0.313047
opp_cum_net_rtg              -0.330177
opp_avg_score_diff_last_10   -0.336407
Name: score_diff, Length: 61, dtype: float64

In [6]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np

features = ["avg_score_comp_last_10", "efg_comp_last_10", "avg_tov_comp_last_10", "pace_mismatch_signed",
            "avg_orb_comp_last_10", "avg_fta_comp_last_10", "is_Home", "rest_days_comp", "net_rtg_comp","home_road_split_comp"]


K = 20
df["score_diff_capped"] = df["score_diff"].clip(-K, K)

target = "score_diff_capped"

train_df = df[df["season"].isin([2023, 2024, 2025])]
test_df  = df[df["season"] == 2026]

#---

X_train = train_df[features]
y_train = train_df[target].astype(float)   # Series

X_test_all = test_df[features]
y_test_all = test_df["score_diff"].astype(float)

mask_all = X_test_all.notna().all(axis=1) & y_test_all.notna()
X_test_all, y_test_all = X_test_all[mask_all], y_test_all[mask_all]

#---

test_df = test_df.copy()
test_df["abs_score_diff"] = test_df["score_diff"].astype(float).abs()

test_under_18 = test_df[test_df["abs_score_diff"] < 18].copy()

X_test_under = test_under_18[features]
y_test_under = test_under_18["score_diff"].astype(float)

test_over_18 = test_df[test_df["abs_score_diff"] >= 18].copy()

X_test_over = test_over_18[features]
y_test_over = test_over_18["score_diff"].astype(float)




# drop rows with missing values
train_mask = X_train.notna().all(axis=1) & y_train.notna()
test_mask  = X_test_all.notna().all(axis=1) & y_test_all.notna()

X_train, y_train = X_train[train_mask], y_train[train_mask]
X_test, y_test   = X_test_all[test_mask], y_test_all[test_mask]

model = XGBRegressor(
    n_estimators=600,
    max_depth=3,
    learning_rate=0.04,
    subsample=0.75,
    colsample_bytree=0.75,
    min_child_weight=20,
    reg_alpha=1.0,
    reg_lambda=3.0,
    objective="reg:pseudohubererror",
    random_state=42,
)

model.fit(X_train, y_train)
preds_under = model.predict(X_test_under)
preds_over = model.predict(X_test_over)
preds_all = model.predict(X_test_all)

print("MAE Low:", mean_absolute_error(y_test_under, preds_under))
print("MAE High:", mean_absolute_error(y_test_over, preds_over))
print("RMSE:", np.sqrt(mean_squared_error(y_test_all, preds_all)))
print("R2:", r2_score(y_test_all, preds_all))



MAE Low: 7.551108585587184
MAE High: 17.72711450494708
RMSE: 13.316553741160398
R2: 0.4158392324166187


In [7]:
features_2026 = pd.read_csv("/Users/mavinjames/Projects/Basketball_Modeling/NCAAB_Sports_Reference_Scraper/data/2026/features_2026_new.csv")
features_2026["date"] = pd.to_datetime(features_2026["date"])

# predict all rows
features_2026["pred_spread"] = model.predict(features_2026[features])

# keep only favored team rows
favored_df = features_2026[features_2026["pred_spread"] > 0].copy()

# dedupe to one row per game
favored_df["team_a"] = favored_df[["school_name","opp_name_abbr"]].min(axis=1)
favored_df["team_b"] = favored_df[["school_name","opp_name_abbr"]].max(axis=1)
favored_df = favored_df.drop_duplicates(["date","team_a","team_b"])

predictions_df = favored_df[["date","school_name","opp_name_abbr","pred_spread", "is_Home"]]


In [ ]:
predictions_df.to_csv('/Users/mavinjames/Projects/Basketball_Modeling/NCAAB_Sports_Reference_Scraper/data/predictions.csv')

In [102]:
features_2026 = pd.read_csv("/Users/mavinjames/Projects/Basketball_Modeling/NCAAB_Sports_Reference_Scraper/data/2026/features_2026_new.csv")
features_2026["date"] = pd.to_datetime(features_2026["date"])
day = pd.Timestamp("2026-01-25")

today_df = features_2026[features_2026["date"] == day].copy()
today_df["pred_spread"] = model.predict(today_df[features])
today_df = today_df[today_df["pred_spread"] > 0].copy()

print(today_df[["date", "school_name", "opp_name_abbr", "pred_spread"]].to_string(index=False))

      date       school_name        opp_name_abbr  pred_spread
2026-01-25            Hawaii Cal State Northridge    14.877108
2026-01-25        Mercyhurst        Chicago State    12.542047
2026-01-25         New Haven   Saint Francis (PA)     5.014743
2026-01-25     South Florida     Florida Atlantic     7.033336
2026-01-25 Southern Illinois           Evansville     6.568174
2026-01-25        Washington               Oregon     7.877153
2026-01-25         Wisconsin  Southern California     8.451253


In [ ]:
import xgboost as xgb
import pandas as pd

# make sure date is datetime and filter
features_2026["date"] = pd.to_datetime(features_2026["date"])
day = pd.Timestamp("2026-01-27")

today_df = features_2026[features_2026["date"] == day].copy()

# predictions
today_df["pred_spread"] = model.predict(today_df[features])
today_df = today_df[today_df["pred_spread"] > 0].copy()

# feature contributions
booster = model.get_booster()
dmat = xgb.DMatrix(today_df[features], feature_names=features)
contribs = booster.predict(dmat, pred_contribs=True)

contribs_df = pd.DataFrame(contribs, columns=features + ["bias"])

# attach id columns so you can filter per game
result = pd.concat(
    [today_df[["date","school_name","opp_name_abbr","pred_spread"]].reset_index(drop=True),
     contribs_df.reset_index(drop=True)],
    axis=1
)

# example: show top 10 contributors for the first game row
i = 8
top10 = result.loc[i, features].sort_values(key=abs, ascending=False).head(10)
print(result.loc[i, ["date","school_name","opp_name_abbr","pred_spread"]])
print(top10)